<a href="https://colab.research.google.com/github/divgup/cs671/blob/master/assignment1_q2(line_dataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving data1.zip to data1.zip
User uploaded file "data1.zip" with length 66915300 bytes


In [0]:
import glob
import cv2
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np

Using TensorFlow backend.


In [0]:
!unzip data1.zip 
images = [cv2.imread(file) for file in glob.glob("data1/*.jpg")]
images = np.array(images)
print(images.shape)






In [0]:
X = images.reshape(images.shape[0],-1).T
#print(X[0][1])
y=np.zeros((96000,1))
for i in range(1,97):
  y[1000*(i-1):(i*1000)] = i-1



```
# This is formatted as code
```

from google.colab import drive
drive.mount('/content/drive')

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X.T, y, test_size=0.33)
print(X_train[1])
print(X_test.shape)
print(y_train.shape)

[12  0  1 ...  0  0  0]
(31680, 2352)
(64320, 1)


In [0]:
#one hot encoding
Y_tr_T = to_categorical(y_train,num_classes=96)
Y_te_T = to_categorical(y_test,num_classes=96)
Y_train = Y_tr_T.T
Y_test = Y_te_T.T
print(Y_train.shape)

(96, 64320)


In [0]:
X_train = X_train/255.0
X_test = X_test/255.0
X_train=X_train.T
X_test=X_test.T
print(X_train.shape)
#print(X_train_flat.shape)

(2352, 64320)


In [0]:
#print(X_train[1][0])
def softmax(a):
  return np.exp(a)/(np.sum(np.exp(a),axis=0,keepdims=True))
def Relu(u):
  u[u<0] = 0
  return u 

In [0]:
parameters = {}
gamma = 0.9
def initialisation():
  num_units_layer1 = X_test.shape[0]
  num_units_layer2 = 196
  num_units_layer3 = 96
  layers = [num_units_layer1,num_units_layer2,num_units_layer3]
  for it in range(1,len(layers)):
    parameters["w"+str(it)] = np.random.rand(layers[it-1],layers[it])*np.sqrt(2/layers[it-1])
    parameters["b"+str(it)] = np.zeros((layers[it],1))
    parameters["prev_v_w"+str(it)] = np.zeros((layers[it-1],layers[it]))
    parameters["prev_v_b"+str(it)] = np.zeros((layers[it],1)) 
  #print(parameters["w2"].shape)
  #print(weights[1])

In [0]:
def calc_loss(Y_train,output):
  loss = np.sum(np.sum(-Y_train*np.log(output),axis=0,keepdims=True),axis=1,keepdims=True)
  return loss/Y_train.shape[1]

In [0]:
layer = {}
def forward_pass(x,num_layers):
  for it in range(num_layers-1):
    #print(parameters["w"+str(it+1)].shape)
    layer["pre_hid"+str(it+1)] = np.dot(parameters["w"+str(it+1)].T,x)+parameters["b"+str(it+1)]
    #print(layer)
    if it < num_layers-2:
      layer["activated"+str(it+1)] = Relu(layer["pre_hid"+str(it+1)])
    else:
      #print(layer["pre_hid"+str(it+1)])
      layer["activated"+str(it+1)] = softmax(layer["pre_hid"+str(it+1)])
      #print(layer["activated"+str(it+1)])
    x = layer["pre_hid"+str(it+1)]          


In [0]:
def drelu(x):
  y = np.ones(x.shape)
  y[x<0] = 0
  return y

In [0]:
def back_prop(X_train,Y_train,num_layers):
  output = layer["activated"+str(num_layers-1)]
  #print(output.shape)
  #print(Y_train.shape)
  grad = {}
  m = X_train.shape[1]
  grad["a"+str(num_layers-1)] = -(Y_train-output)/m
  grad["h"+str(num_layers-2)] = np.dot(parameters["w"+str(num_layers-1)],grad["a"+str(num_layers-1)])
  #grad["w"+str(len(layers)-1)] = np.dot(output-Y_train,x[pre_hid+str(len(layers)-2)].T)
  for it in range(1,num_layers-1):
    Indicator = drelu(layer["pre_hid"+str(num_layers-it-1)])
    grad["a"+str(num_layers-1-it)] = np.multiply(grad["h"+str(num_layers-it-1)],Indicator) 
    grad["h"+str(num_layers-it-2)] = np.dot(parameters["w"+str(num_layers-it-1)],grad["a"+str(num_layers-1-it)])
  grad["w1"] = np.dot(grad["a1"],X_train.T)
  grad["b1"] = np.sum(grad["a"+str(1)],axis=1,keepdims=True)
  for it in range(1,num_layers-1):  
    grad["w"+str(it+1)] = np.dot(grad["a"+str(it+1)],layer["pre_hid"+str(it)].T) 
    grad["b"+str(it+1)] = np.sum(grad["a"+str(it+1)],axis=1,keepdims=True)
  return grad  

In [0]:
def weight_update(num_layers,alpha,grad):
  for i in range(num_layers-1):
    parameters["w"+str(num_layers-i-1)] = parameters["w"+str(num_layers-i-1)] - (alpha*grad["w"+str(num_layers-i-1)]).T - (gamma*parameters["prev_v_w"+str(num_layers-i-1)])
    parameters["b"+str(num_layers-i-1)] = parameters["b"+str(num_layers-i-1)] - (alpha*grad["b"+str(num_layers-i-1)]) - (gamma*parameters["prev_v_b"+str(num_layers-i-1)])
    parameters["prev_v_w"+str(num_layers-i-1)] = (alpha*grad["w"+str(num_layers-i-1)]).T + (gamma*parameters["prev_v_w"+str(num_layers-i-1)])
    parameters["prev_v_b"+str(num_layers-i-1)] = (alpha*grad["b"+str(num_layers-i-1)]) + (gamma*parameters["prev_v_b"+str(num_layers-i-1)])
  return parameters  

In [0]:
from sklearn.utils import shuffle
import math
alpha = 0.01
num_layers = 3
cost=[]
num_itr = 1000
num_hidden_layers = 2
initialisation()
#forward_pass(X_train,num_layers)
num_epochs = 1000
batch_size = 1000
no_batch_per_epoch = math.floor(X_train.shape[0]/batch_size)
print(X_train.shape,Y_train.shape)
#print(layer["activated"+str(num_layers-1)])
for it in range(num_itr):
  forward_pass(X_train,num_layers)
  output = layer["activated"+str(num_layers-1)]
  grad = back_prop(X_train,Y_train,num_layers)
  #print(parameters["b1"].shape)
  weight_update(num_layers,alpha,grad)
  if it%10==0:
    cost.append(calc_loss(Y_train,output))
    print(calc_loss(Y_train,output))   

print(parameters["b1"],parameters["w1"])    

In [0]:
print(parameters["b1"],parameters["w1"])  